In [1]:
import os
import time
import re
import pickle
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from bs4 import BeautifulSoup


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#driver = webdriver.Chrome()

options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")
driver = webdriver.Chrome()

# DURING THIS TIME YOU NEED TO CHANGE THE CURRENCY FROM YEN TO DOLLARS AND TIME TO UST MANUALLY IN THE WEBSITE SETTINGS
driver.get('https://vt.poi.cat/settings')
time.sleep(30)

#year = input('Please enter the year you want to collect')
year = '2022'

sel_date_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", "//button[contains(@class, '!rounded-full ng-tns-c1284214496-13 mdc-button mdc-button--outlined mat-mdc-outlined-button mat-unthemed mat-mdc-button-base')]")))
sel_date_button.click()
time.sleep(1)

sel_year_cont_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", "//button[contains(@class, 'mat-calendar-period-button mdc-button mat-mdc-button mat-unthemed mat-mdc-button-base')]")))
sel_year_cont_button.click()
time.sleep(1)


sel_year_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", f"//button[contains(@aria-label, '{year}')]")))
sel_year_button.click()
time.sleep(1)


first_month_year = 'January ' + f'{year}'
sel_month_button =  WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", f"//button[contains(@aria-label, '{first_month_year}')]")))
sel_month_button.click()
time.sleep(1)

first_of_month = 'January 1, ' + f'{year}'
sel_first_button =  WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", f"//button[contains(@aria-label, '{first_of_month}')]")))
sel_first_button.click()
time.sleep(1)

sel_year_cont_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", "//button[contains(@class, 'mat-calendar-period-button mdc-button mat-mdc-button mat-unthemed mat-mdc-button-base')]")))
sel_year_cont_button.click()
time.sleep(1)


sel_year_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", f"//button[contains(@aria-label, '{year}')]")))
sel_year_button.click()
time.sleep(1)

last_month_year = 'December ' + f'{year}'
sel_month_button =  WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", f"//button[contains(@aria-label, '{last_month_year}')]")))
sel_month_button.click()
time.sleep(1)

last_of_month = 'December 31, ' +  f'{year}'
sel_sec_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", f"//button[contains(@aria-label, '{last_of_month}')]")))
ActionChains(driver).key_down(Keys.SHIFT).click(sel_sec_button).key_up(Keys.SHIFT).perform()

time.sleep(5)


every_hundreth = 0
element = driver.find_element('xpath', '//body')

while True:
    
    # Element.send_keys(Keys.PAGE_DOWN)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(2)
    # Loading every 100 ticks drastically reduces collection time
    every_hundreth += 1
    # try:
    #     WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", "//img[contains(@loading, 'lazy')]")))
    # except:
    #     break  # not more posts were loaded - exit the loop
    
    
    if every_hundreth%10 == 0:
        
        stop_element = driver.find_elements("xpath", "//div[contains(@class, 'mat-h3 font-medium ng-star-inserted')]")[-1].text
        print(stop_element)
        
        if f'{year}/01/01' in stop_element:
            
            elems = driver.find_elements("xpath", "//a[@href]")
            print("The number of elements is: " + str(len(elems)))
            print("Now collecting Links")
            holo_raw_data = [elem.get_attribute("href") for elem in elems if '/youtube-stream' in elem.get_attribute("href")]
            #print(filtered_urls)
            print("The length of the raw data is: " + str(len(holo_raw_data)))
            #pickle.dump(holo_raw_data, open(f'vstats_URL_{year}.pkl','wb'))
            year_col = True
            print(f'YEAR {year} COLLECTED')
            
            # Removes duplicates, but does not preserve the order
            # res = [*set(holo_raw_data)]
            # print(len(res))
            
            # Removes duplicates while preserving order of original list since we want them chronologically
            seen = set()
            seen_add = seen.add
            holo_ref_data = [x for x in holo_raw_data if not (x in seen or seen_add(x))]
            
            print(len(holo_ref_data))
            driver.quit()

In [19]:
pickle.dump(holo_raw_data, open(f'vstats_URL_RAW_2022.pkl','wb'))
pickle.dump(holo_ref_data, open(f'vstats_URL_2022.pkl','wb'))

In [9]:
driver = webdriver.Chrome()
df = pd.DataFrame(columns=['URL', 'Video_ID', 'Video_Title', 'Creator', 'Avg_viewers', 'Max_viewers', 'SuperChat_total', 'Stream_Start_Date_UST', 'Stream_Start_Time_UST', 'duration'])

# DURING THIS TIME YOU NEED TO CHANGE THE CURRENCY FROM YEN TO DOLLARS AND TIME TO UST MANUALLY IN THE WEBSITE SETTINGS
driver.get('https://vt.poi.cat/settings')
time.sleep(30)

#year = input('Please enter the year you want to colect')

#holostats = pickle.load(open(f'holostats_URL_2022_REF.pkl','rb'))
#holostats = pickle.load(open(f'\\Pickled data\\holostats_URL_{year}_REF.pkl','rb'))
holostats = pickle.load(open('vstats_URL_2022.pkl','rb'))

data_list = []

for x in range(len(holostats)):

    url = holostats[x]
    driver.get(url)

    temp = []
    temp.append(url)
    time.sleep(5)
    page_sour = driver.page_source
    doc = BeautifulSoup(page_sour, "lxml")

    vid_id = str(url).split('/')
    vid_id = vid_id[-1]
    print(vid_id, end =" ")
    temp.append(vid_id)

    try:
        #title = doc.find('title')
        title = driver.title
        title = str(title)
        #title = title[:-12]
        print(title, end =" ")
        temp.append(title)
    except:
        temp.append(" ")
        continue

    try:
        creator = doc('span', {'class': 'ml-2 align-middle'})[0].text
        print(creator, end =" ")
        temp.append(creator)
    except:
        temp.append(" ")
        continue

    try:
        avg_max_view = doc('div', {'class': 'mat-headline-5 m-0'})[0].text
        avg_max_view = avg_max_view.split('/')

        avg_view = avg_max_view[0]
        avg_view = avg_view[1:-1]
        print(avg_view, end =" ")
        temp.append(avg_view)

        max_view = avg_max_view[1]
        max_view = max_view[1:-1]
        print(max_view, end =" ")
        temp.append(max_view)
    except:
        temp.append(" ")
        temp.append(" ")
        continue

    try:   
        #sup_chat_tot = doc.find_all(text=re.compile("\¥.*"))
        sup_chat_tot = doc.find_all(text=re.compile("\$.*"))
        print(sup_chat_tot, end =" ")
        temp.append(sup_chat_tot[0][3:]) 
    except:
        temp.append(" ")
        continue
        
    try:
        stream_date_time = doc('span', {'class': 'mat-body-1'})[0].text
        date_time_split = stream_date_time.split(' ')
        print(date_time_split[2], end=" ")
        temp.append(date_time_split[1])
        print(date_time_split[2], end=" ")
        temp.append(date_time_split[2])
    except:
        temp.append(" ")
        continue
        
    try:
        duration = doc('span', {'class': 'mat-body-1'})[2].text
        print(duration, end=" ")
        temp.append(duration)
    except:
        temp.append(" ")
        continue
    
    print(" ")
    data_list.append(temp)
    df.loc[len(df)] = temp

#df.to_excel(f'output_{year}.xlsx')
df.to_excel('output_2022_TEST.xlsx')
#pickle.dump(df, open(f'holostats_{year}.pkl','wb'))
driver.quit()
#print(doc.prettify())
#title = doc.find_all('span')
#print(title)

iSZfA1X-lzE 【UNARCHIVED KARAOKE】 Singing til the new year!!! | vtstats Ceres Fauna 13,542 16,155 [' US$2,788.90 ', ' $2,605.82 ', ' $1,737.54 ', ' $266.15 ', ' $240.00 ', ' $182.32 ', ' $72.25 ', ' $43.78 ', ' $24.39 ', ' $20.05 ', ' $19.34 '] 05:53:07 05:53:07  02:14:25   
b4ZvEgeyjbI 【🎍2023年🎍】書道「段」持ちがマウスで挑戦！！デジタル書き初め！！【天音かなた/ホロライブ】 | vtstats 天音かなた 12,305 14,550 [' US$2,635.44 ', ' $2,635.44 ', ' $2,109.56 ', ' $330.00 ', ' $97.34 ', ' $44.53 ', ' $32.00 ', ' $14.85 ', ' $5.00 ', ' $1.28 ', ' $0.88 '] 05:00:56 05:00:56  01:58:31   
iueP6WtC6aQ 【Valorant】年明け早々プラチナに行きたい！ウィスパー声縛りvalorant【ロボ子さん/ホロライブ】 | vtstats ロボ子さん 1,684 2,042 [' US$175.40 ', ' $155.23 ', ' $123.26 ', ' $20.00 ', ' $6.10 ', ' $5.13 ', ' $0.74 '] 04:44:19 04:44:19  04:27:35   
hGsUovQ4N7Y NEW YEAR CHITCHAT & Rewatching my official program appearance!!! あけおめ！！！　#kfp #キアライブ ` | vtstats Takanashi Kiara 2,984 4,309 [' US$4,601.01 ', ' $4,601.01 ', ' $3,394.34 ', ' $267.47 ', ' $222.64 ', ' $175.00 ', ' $155.07 ', ' $126.45 '

In [34]:
data_list

[]

In [36]:
url = 'https://vt.poi.cat/youtube-stream/iSZfA1X-lzE'
driver = webdriver.Chrome()
driver.get(url)